In [64]:
!pip install langchain #The main library to use langchain components
!pip install langchain_google_genai #It is langchain package to connect with gemini models through langchain.
!pip install langchain_classic

In [65]:
from google.colab import userdata
gem = userdata.get('Gemini')

import os
os.environ["GOOGLE_API_KEY"] =gem
# We have setuped the environment variable for accessing gemini models

In [66]:
import langchain
import langchain_classic #To access chain
# langchain_classic is used to support the old chain-based APIs like LLMChain and ConversationChain. It helps old projects keep working after LangChain was divided into smaller packages like langchain_core and langchain_community.

from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser # For Output Parser

## Problem statement

The main objective of this case study is to clearly understand and compare three different workflows in LangChain in an effective and practical way.

To achieve this, we will build a simple LLM application that takes a story as input and generates 3 to 4 meaningful questions to test comprehension and thinking ability.

In [67]:
# Step 1 = Create a prompt template
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [68]:
runn1_cpt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a 10 years of experience in writing and generating a story for children's"),
                                  HumanMessagePromptTemplate.from_template("Based on this genre: {genre} generate a story")])

In [69]:
# Step 2 = Set LLM Model
runn2_model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite")

In [70]:
# Step 3 = Create Output
runn3_strp = StrOutputParser()

## Types of Runnables

### 1. Runnable Sequence

In [71]:
from langchain_core.runnables import RunnableSequence

# We import RunnableSequence from langchain_core.runnables to combine multiple steps (like prompt → model → output parser) into a single structured pipeline.

# It allows us to execute steps in sequence using the modern workflow design of LangChain, making the application more modular, clean, and scalable.

In [72]:
# Functions is like a runnables (Process[input] generate output)


In [73]:
sequential_chain = RunnableSequence(runn1_cpt,runn2_model,runn3_strp) # Another way to mension = runn1_cpt|runn2_model|runn3_strp

### For one input variable = {"genre":"Adventure"}

In [74]:
response = sequential_chain.invoke({"genre":"Adventure"}) # Give input as dict only
# Runnable = Take Single Input --> Return Single Output
# If we want multiple output so for that we need to make dictionary.

In [75]:
print(response)

Barnaby Button was an adventurer. Not the kind who battled dragons or discovered lost cities, mind you. Barnaby's adventures were usually confined to the wild, untamed jungle of his own backyard.

One sunny morning, a glint of something unusual caught Barnaby's eye near the ancient oak tree. It was a tiny, shimmering feather, unlike any he'd ever seen. It was the colour of a rainbow after a summer shower, with hints of emerald green and sapphire blue.

"A treasure!" Barnaby whispered, his heart thumping with excitement. He carefully picked it up. It felt soft and strangely warm in his fingers. "This must belong to a very special bird!"

His mission was clear: find the owner of the rainbow feather. He grabbed his trusty magnifying glass (essential for any serious explorer) and his trusty sidekick, a fluffy, ginger cat named Marmalade, who mostly enjoyed napping in sunbeams but occasionally batted at interesting bugs.

Barnaby started his expedition at the base of the oak. He scanned the

### For 2 input variables = {"genre":"Adventure","years":10}

In [76]:
runn2_cpt = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a {years} years of experience in writing and generating a story for children's"),
                                  HumanMessagePromptTemplate.from_template("Based on this genre: {genre} generate a story")])

In [77]:
sequential_chain_2 = RunnableSequence(runn2_cpt,runn2_model,runn3_strp) # Another way to mension = runn2_cpt|runn2_model|runn3_strp

In [78]:
response_2 =sequential_chain_2.invoke({"genre":"Adventure","years":10})
# here we pass 2 inputs

In [79]:
print(response_2)

Barnaby Button was not your average ten-year-old. While other kids were busy with video games and soccer practice, Barnaby was busy with maps. Not just any maps, mind you. He collected them, traced them, and dreamed of the places they represented. His bedroom walls were a tapestry of ancient cartography, a testament to his burning desire for adventure.

One rainy Tuesday, while rummaging through his grandfather’s dusty attic, Barnaby stumbled upon a peculiar rolled-up parchment tucked away in an old sea chest. It wasn't a map of any place he recognized. Instead, it was a series of intricate drawings, strange symbols, and a single, faded inscription in a language he couldn't decipher.

"What in the world is this?" he muttered, his heart thrumming with a familiar excitement.

He spent days poring over the parchment, comparing the symbols to ancient alphabets he found in his books. Finally, a breakthrough! The symbols weren't letters, but a code. And the inscription, once deciphered, read

In [80]:
# Whenever we are accessing a single model components (use invoke) dont need to pass dictionary as a variable data.
# But whenever we are using runnables sequence at that time, if we want to pass multiple input, then we use dictionary because it acts like this =
# sequential_chain_2.invoke(input = {"genre":"Adventure","years":10})

### One condition :

If we pass an extra variable like "x": 10 in :
sequential_chain_2.invoke({"genre":"Adventure","years":10,"x":10})
#### and "x" is not used in the prompt template, it will not give an error.

- LangChain simply ignores the extra variable by default and only uses the variables that are defined in the prompt template.

- So unused inputs are automatically skipped.

## Problem statement:

LLM Workflow:

- Use the LLM to generate a story based on given inputs (e.g., genre, theme, years).

- Pass the generated story as input to the same or another LLM.

- The LLM analyzes the story and provides a rating (e.g., out of 10) based on quality, creativity, and coherence.

Simple Flow:
LLM → Generate Story → Pass Story → LLM → Give Rating

In [81]:
# chatpromptTemplate_1 --> LLM --> story --> outputparser --> story --> chatpromptTemplate_2 --> LLM --> ratings

In [82]:
# Create Prompt_Template for (creating a story)
cpt_1 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a {years} years of experience in writing and generating a story for children's"),
                                  HumanMessagePromptTemplate.from_template("Based on this genre: {genre} generate a story")])

# Create Prompt_Template for (to generate ratings)
cpt_2 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a 10 years of experience in giving ratings on a story"),
                                  HumanMessagePromptTemplate.from_template("Based on this story: {story} rate it out of 5 where 5 is the best and 1 is the worst ")])

In [83]:
# (years,genre) 2_input_variable-->ChatPromptTemplate[runn_cpt_1]:Prompt-->LLM--return[AI message]-->StringOutputParser-->Return[Text] NEXT👇
# Return[Text] go as -->(Story) 1_input_variable-->ChatPromptTemplate[runn_cpt_2]:Prompt-->LLM--return[AI message]-->StringOutputParser-->rating

In [88]:
from google.colab import userdata
gem_lite = userdata.get('gemini_lite')

import os
os.environ["GOOGLE_API_KEY"] =gem_lite
# We have setuped the environment variable for accessing gemini models

In [89]:
# Step 2 = Set LLM Model
model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite")

In [90]:
seq_chain_1 = cpt_1|model|runn3_strp|cpt_2|model|runn3_strp # Using Sequence Chain

In [91]:
response_rating = seq_chain_1.invoke({"years":20,"genre":"adventure"})

In [92]:
print(response_rating)

Here's my rating of the story "Barnaby Button":

**Rating: 4.5 out of 5**

**Reasoning for the rating:**

This is a charming and well-crafted adventure story for young readers. Here's a breakdown of why it earns a high score:

*   **Engaging Protagonist:** Barnaby is an instantly likable and relatable character. His imaginative nature and thirst for adventure make him the perfect hero for this kind of tale. The contrast with his peers is a nice touch that grounds him in childhood reality.
*   **Magical Premise:** The discovery of the "Whispering Feather" is a classic and effective fantasy trope. The immediate transportation to the jungle is a great hook that propels the story forward with excitement.
*   **Clear and Satisfying Plot Arc:** The story follows a classic adventure structure: discovery, journey, challenges, and reward. Each element is present and well-executed.
*   **Creative Challenges:** The monkey encounter and the river crossing are age-appropriate and clever challenges 

In [93]:
runnable_sequence= RunnableSequence(cpt_1,model,runn3_strp,cpt_2,model,runn3_strp)

In [94]:
runnable_sequence.invoke({"years":20,"genre":"adventure"})

# Here cpt_2 automatically able to undersand (Entire output coming from [cpt_1,runn2_model,runn3_strp] chain) and giving to the variable {story})
# Because this entire chain is giving only one output that is a single text
# That's why we no need to pass story as an output

'Here\'s my rating of Barnaby Button\'s adventure:\n\n**Rating: 4.5/5**\n\n**Reasoning (with 10 years of experience in mind):**\n\nThis is a truly delightful story with a lot of charm and potential. Here\'s a breakdown of why it earns a high rating:\n\n*   **Character Development (4.5/5):** Barnaby is immediately likeable. His description – "knobbly knees," "unruly ginger hair," and his backyard adventures – makes him relatable and endearing, especially to a younger audience. The story effectively establishes his yearning for adventure and his optimistic spirit, even when faced with minor setbacks. His Mum\'s gentle encouragement also adds a nice touch.\n\n*   **Plot and Pacing (4/5):** The plot unfolds at a good pace. The discovery of the locket is a strong hook, and the subsequent investigations at the oak tree and the well create a sense of mystery and anticipation. The resolution, while unexpected, is satisfying and fits the theme of the story. The story doesn\'t linger too long on

In [95]:
# Use another LLM : runnablepassthorug

### RunnablePassthrough = It is a primitive runnable in LangChain that simply returns the input as it is.

👉 It does not modify or process the data.

👉 Whatever input you give, it sends the same input as output.

- In simple words:
RunnablePassthrough = Input goes in → Same input comes out.

In [96]:
from langchain_core.runnables import RunnablePassthrough


In [97]:
rs =RunnablePassthrough()
# It return as output as input
# that input we can first assigned it to an another variable
# Input --> assigned [Variable]

In [98]:
# Example:
rs.invoke("2")

'2'

In [99]:
seq_chain_3 = RunnablePassthrough.assign(story = RunnableSequence(cpt_1,model,runn3_strp)) | cpt_2 | model | runn3_strp

- First, we generate the story.

- Then, we use that same story in the next prompt (for rating, questions, etc.).

- RunnablePassthrough.assign() helps us keep the original input and add the generated story to it.

#### We do this to build a multi-step workflow where:

Input → Story Generated → Story Used Again → Final Result

- It helps us connect steps smoothly using the modern pipeline design of LangChain.

In [100]:
seq_chain_3.invoke({"years":20,"genre":"adventure"})

'This is a delightful story with a lot of charm! Here\'s my rating based on my experience:\n\n**Rating: 4.5 out of 5**\n\nHere\'s a breakdown of why:\n\n**Strengths:**\n\n*   **Imaginative Protagonist:** Barnaby is a wonderfully conceived character. His passion for maps, especially unusual ones, is unique and engaging for a ten-year-old. This immediately sets him apart and makes him relatable to anyone who has had a strong, quirky interest.\n*   **Captivating Premise:** The "Whispering Isle" concept is immediately intriguing. The idea of a map leading to a place that communicates through sound is a fantastic hook.\n*   **Charming Sidekick:** Professor Wigglesworth is the perfect companion. His "portly" nature and talent for sniffing out snacks and trouble add humor and a touch of groundedness to Barnaby\'s fantastical quest. His "woof" of readiness is a great touch.\n*   **Well-Paced Adventure:** The story moves at a good pace. We get Barnaby\'s initial discovery, his persuasive approa

In [101]:
# Create Prompt_Template for (creating a story)
cpt_3 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a {years} years of experience in writing and generating a story for children's"),
                                  HumanMessagePromptTemplate.from_template("Based on this genre: {genre} generate a story")])

# Create Prompt_Template for (to generate ratings)
cpt_4 = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template("You are having a {exp} years of experience in giving ratings on a story"),
                                  HumanMessagePromptTemplate.from_template("Based on this story: {story} rate it out of 5 where 5 is the best and 1 is the worst ")])

In [102]:
sequential_chain_4 = RunnableSequence(cpt_3,model,runn3_strp,cpt_4,model,runn3_strp) # Another way to mension = runn2_cpt|runn2_model|runn3_strp
# cpt_3(years,genre) and cpt_4(exp,story) both having 2 inputes

In [104]:
seq_chain_4 = RunnablePassthrough.assign(story = RunnableSequence(cpt_3,model,runn3_strp))

In [105]:
response_4 = seq_chain_4.invoke({"years":20,"genre":"adventure"})

In [106]:
print(response_4)

{'years': 20, 'genre': 'adventure', 'story': "Barnaby Button loved adventure. Not the kind of adventure where you wrestle a dragon or discover a hidden treasure (though he wouldn't say no to that!). Barnaby's adventures were usually much smaller, and much more… ordinary.\n\nOne sunny Tuesday morning, Barnaby woke with a tingle in his toes. Today, he declared to his teddy bear, Professor Snuggles, was the day he would conquer the Great Unknown. The Great Unknown, in Barnaby's world, was the mysterious patch of overgrown weeds at the very back of his garden, a place his mum always told him to stay away from.\n\nHe pulled on his trusty explorer boots (wellington boots, really) and his brave knight helmet (a colander). Professor Snuggles, tucked safely into Barnaby's backpack, gave a silent nod of encouragement.\n\nCreeping out the back door, Barnaby surveyed his mission. The Great Unknown loomed before him, a jungle of towering dandelions and whispering grass. His heart thumped like a dru

In [107]:
RunnablePassthrough.assign(exp =lambda response:response_4["years"])

RunnableAssign(mapper={
  exp: RunnableLambda(lambda response: response_4['years'])
})

In [113]:
from google.colab import userdata
gemini = userdata.get('gemini')

import os
os.environ["GOOGLE_API_KEY"] =gemini

In [114]:

model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite")

In [115]:
seq_chain_5 = RunnablePassthrough.assign(story = RunnableSequence(cpt_3,model,runn3_strp)) | RunnablePassthrough.assign(exp =lambda response:response_4["years"])

In [116]:
response_5 = seq_chain_5.invoke({"years":20,"genre":"adventure"})

In [117]:
response_5

{'years': 20,
 'genre': 'adventure',
 'story': 'Barnaby Button, a boy with knees perpetually smudged with dirt and a heart that thumped like a hummingbird\'s wings, lived in a town that was, to put it mildly, rather ordinary. The most exciting thing that happened was when Mrs. Higgins\' cat, Bartholomew, got stuck up the oak tree for the third time that month. But Barnaby dreamed of more. He dreamed of hidden treasures, of roaring beasts, and of lands painted with colors no one had ever seen.\n\nOne sun-drenched afternoon, while exploring the dusty attic of his grandmother’s house, Barnaby stumbled upon an old, leather-bound book. Its pages were brittle, and the ink had faded to a whisper, but the title, etched in gold that still gleamed faintly, read: "The Chronicles of the Whispering Isles."\n\nAs Barnaby opened the book, a gust of wind, smelling of salt and something sweet like mangoes, swept through the attic, though all the windows were shut tight. The pages began to shimmer, and 

In [118]:
print(response_5.keys())

dict_keys(['years', 'genre', 'story', 'exp'])


In [120]:
Final_chain = RunnablePassthrough.assign(story=RunnableSequence(cpt_3,model,runn3_strp)) | RunnablePassthrough.assign(exp = lambda response:response_4["years"]) | RunnableSequence(cpt_4,model,runn3_strp)

In [121]:
Final_response = Final_chain.invoke({"years":20,"genre":"adventure"})

In [122]:
print(Final_response)

As an experienced reviewer of stories, I've had the pleasure of delving into countless narratives. Barnaby Button's adventure in the Whispering Woods is a charming and well-crafted tale that taps into the pure magic of childhood imagination.

Here's my rating and breakdown:

**Rating: 4.5 out of 5**

**Breakdown:**

*   **Concept and Imagination (5/5):** The core concept of "Adventure Boots" and the "Whispering Woods" is brilliant. It perfectly captures the essence of a child's boundless imagination and their ability to transform the mundane into the extraordinary. The way Barnaby's backyard becomes a portal to a magical realm is beautifully depicted.
*   **Character Development (4/5):** Barnaby is an endearing protagonist. His unwavering belief in his Adventure Boots and his patient teddy bear, Professor Snuggles, makes him instantly likable. While Professor Snuggles is a silent companion, his presence adds to Barnaby's world. The Guardian of the Woods serves its purpose effectively a

In [130]:
Final_dict_output= RunnablePassthrough.assign(story=RunnableSequence(cpt_3,model,runn3_strp)) | RunnablePassthrough.assign(exp = lambda response:response_4["years"]) | RunnablePassthrough.assign(output = cpt_4 | model | runn3_strp)

In [131]:
dict_Final_output = Final_dict_output.invoke({"years":20,"genre":"adventure"})

In [132]:
dict_Final_output.keys()

dict_keys(['years', 'genre', 'story', 'exp', 'output'])

In [133]:
dict_Final_output

{'years': 20,
 'genre': 'adventure',
 'story': '## The Whispering Waterfall and the Lost Locket\n\nBarnaby Button was a boy whose pockets always seemed to be full of interesting things: smooth grey pebbles, a particularly shiny beetle wing, and sometimes, if he was lucky, a forgotten jellybean. Barnaby loved adventures, the kind that started with a hunch and ended with a discovery.\n\nOne sunny afternoon, while exploring the edge of the Whispering Woods, Barnaby heard a faint, tinkling sound. It was like tiny bells dancing on the wind, but it wasn\'t coming from the trees. He followed the sound, pushing aside ferns as tall as his shoulders, his heart thumping with that familiar thrill of the unknown.\n\nThe tinkling grew louder, leading him deeper into the woods until he stumbled upon a small, moss-covered clearing. In the center, a tiny waterfall, no bigger than his outstretched arms, cascaded down smooth, grey rocks. And there, tangled in the delicate spray, was a beautiful, heart-sh